In [1]:
import pandas as pd
import altair as alt
from datetime import datetime, timedelta
from urllib import request
import json


In [2]:
# fetch & enable a Spanish timeFormat locale.
with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-ES.json') as f:
  es_time_format = json.load(f)
alt.renderers.set_embed_options(timeFormatLocale=es_time_format)


covid_total = 'https://raw.githubusercontent.com/mariorz/covid19-mx-time-series/master/data/covid19_confirmed_mx.csv'
covid_total = pd.read_csv(covid_total, index_col=0)
covid_total = covid_total.loc['Colima','17-03-2020':'18-11-2020']
covid_total = pd.DataFrame(covid_total)
covid_total.index = pd.to_datetime(covid_total.index, format='%d-%m-%Y')
covid_total['Diarios'] = covid_total.Colima.diff()
covid_total_7 = covid_total.rolling(window=7).mean() 

# Suspects
covid_suspects = 'https://raw.githubusercontent.com/mariorz/covid19-mx-time-series/master/data/covid19_suspects_mx.csv'
covid_suspects = pd.read_csv(covid_suspects, index_col=0)
covid_suspects = covid_suspects.loc['Colima','17-03-2020':'18-11-2020']
covid_suspects = pd.DataFrame(covid_suspects)
covid_suspects.index = pd.to_datetime(covid_suspects.index, format='%d-%m-%Y')
#covid_suspects['Diarios'] = covid_suspects.Colima.diff()
covid_suspects_7 = covid_suspects.rolling(window=7).mean() 

# Hospitalized by admission date
covid_hospitalized = 'https://raw.githubusercontent.com/mariorz/covid19-mx-time-series/master/data/full/by_hospital_state/hospitalized_confirmed_by_admission_date_mx.csv'
covid_hospitalized = pd.read_csv(covid_hospitalized, index_col=0)
covid_hospitalized = covid_hospitalized.loc['Colima','17-03-2020':'18-11-2020']
covid_hospitalized = pd.DataFrame(covid_hospitalized)
covid_hospitalized.index = pd.to_datetime(covid_hospitalized.index, format='%d-%m-%Y')
covid_hospitalized['Diarios'] = covid_hospitalized.Colima.diff()
covid_hospitalized_7 = covid_hospitalized.rolling(window=7).mean() 

# Deaths by death date

deaths_total = 'https://raw.githubusercontent.com/mariorz/covid19-mx-time-series/master/data/full/by_hospital_state/deaths_confirmed_by_death_date_mx.csv'
deaths_total = pd.read_csv(deaths_total, index_col=0)
deaths_total = deaths_total.loc['Colima','17-03-2020':'23-11-2020']
deaths_total = pd.DataFrame(deaths_total)
deaths_total.index = pd.to_datetime(deaths_total.index, format='%d-%m-%Y')
deaths_total['Diarios'] = deaths_total.Colima.diff()
deaths_total_7 = deaths_total.rolling(window=7).mean() 


In [20]:
%run urban_theme.py

# Confirmed
daily_confirmed = alt.Chart(covid_total.reset_index()).mark_area(fillOpacity=0.40).encode(
    alt.X('index:T', title = " "),
    alt.Y('Diarios:Q', title = " ")
).properties(
    title = "Confirmados por día",
    width = 300,
    height = 300
)

daily_confirmed_7 = alt.Chart(covid_total_7.reset_index()).mark_line(color="red", size=2).encode(
    alt.X('index:T', title = " "),
    alt.Y('Diarios:Q', title = " "),
).properties(
    width = 300,
    height = 300
)

# Suspects
daily_suspects = alt.Chart(covid_suspects.reset_index()).mark_area(fillOpacity=0.4).encode(
    alt.X('index:T', title = " "),
    alt.Y('Colima:Q', title = " ")
).properties(
    title = "Sospechosos por día",
    width = 300,
    height = 300
)

daily_suspects_7 = alt.Chart(covid_suspects_7.reset_index()).mark_line(color="red", size=2).encode(
    alt.X('index:T', title = " "),
    alt.Y('Colima:Q', title = " "),
).properties(
    width = 300,
    height = 300
)

# Hospitalized
daily_hospitalized = alt.Chart(covid_hospitalized.reset_index()).mark_area(fillOpacity=0.40).encode(
    alt.X('index:T', title = " "),
    alt.Y('Diarios:Q', title = " ")
).properties(
    title = "Hospitalizados por día",
    width = 300,
    height = 300
)

daily_hospitalized_7 = alt.Chart(covid_hospitalized_7.reset_index()).mark_line(color="red", size=2).encode(
    alt.X('index:T', title = " "),
    alt.Y('Diarios:Q', title = " "),
).properties(
    width = 300,
    height = 300
)

#Deaths
daily_deaths = alt.Chart(deaths_total.reset_index()).mark_area(fillOpacity=0.40).encode(
    alt.X('index:T', title = " "),
    alt.Y('Diarios:Q', title = " ")
).properties(
    title =  "Defunciones por día",
    width = 300,
    height = 300
)

deaths_7 = alt.Chart(deaths_total_7.reset_index()).mark_line(color="red", size=2).encode(
    alt.X('index:T', title = " "),
    alt.Y('Diarios:Q', title = " "),
).properties(
    width = 300,
    height = 300,
)

confirmed = daily_confirmed+daily_confirmed_7
suspects = daily_suspects+daily_suspects_7
hospitalized = daily_hospitalized+daily_hospitalized_7
deaths = daily_deaths+deaths_7

colima = confirmed | suspects | hospitalized | deaths

In [21]:
colima_data = colima.properties(
    title={
        "text":["El COVID-19 en Colima"],
        "subtitle": ["Datos del 17 de marzo al 23 de noviembre de 2020.", " "],
    }
)

In [22]:

alt.concat(colima_data).properties(
    title=alt.TitleParams(
        ['Fuente: Elaboración propia con datos del repositorio covid19-mx-time-series(@mariorz). Linea roja representa el promedio movil de 7 días.', 'Elaborado por @jballesterosc_'],
        baseline='bottom',
        orient='bottom',
        anchor='end',
        fontWeight='normal',
        fontSize=10
    )
)

alt.ConcatChart(...)